In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import json
import time
from datetime import datetime
from datetime import timedelta
from typing import List

from tqdm import tqdm

In [2]:
df_all =  pd.read_csv(
    "./sentiment_data/data_qa.csv",
    index_col=0,
    dtype={0:'str'}
)

In [3]:
return_path = "./hackathon_data/company_prices_returns/"

In [4]:
# df_returns = pd.read_csv(
#     return_path + company + "_adj_close.csv",
#     date_parser=lambda x: datetime.strptime(x, "%Y-%m-%d"),
#     index_col=0
# )
# # date = datetime.strptime(date, "%Y/%m/%d")
# for i in range(1000):
#     try:
#         nearest_return = df_returns.loc[date + timedelta(i)]
#         break
#     except KeyError:
#         pass

In [5]:
def load_return(
    company: str,
    date: str,
    tol: int = 1000
) -> float:
    df_returns = pd.read_csv(
        return_path + company + "_adj_close.csv",
        date_parser=lambda x: datetime.strptime(x, "%Y-%m-%d"),
        index_col=0
    )
    date = datetime.strptime(date, "%Y/%m/%d")
    for i in range(tol):
        try:
            return df_returns.loc[date + timedelta(i)]["Returns"], date + timedelta(i)
            break
        except KeyError:
            pass
    return None, None

In [6]:
load_return("AAPL", "2030/1/1")

(None, None)

In [7]:
df_with_returns = df_all.copy()
df_with_returns["nearest_day_return"] = None
df_with_returns["nearest_day"] = None

In [8]:
for report_id in tqdm(df_all.index):
#     print(report_id)
    index = report_id.index("_")
    company = report_id[:index]
#     print(f"Processing company: {company}")
    report_date = df_all.loc[report_id].Time
    nearest_day_return, nearest_day = load_return(
        company, report_date
    )
    df_with_returns["nearest_day_return"][report_id] = nearest_day_return
    df_with_returns["nearest_day"][report_id] = nearest_day

  0%|          | 0/12379 [00:00<?, ?it/s]/Users/tianyudu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/tianyudu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 12379/12379 [03:37<00:00, 56.79it/s]


In [9]:
df_with_returns.to_csv("./sentiment_data/LMD_data_all_returns.csv")